<a href="https://colab.research.google.com/github/monocleface/huggingface-tutorials/blob/main/sample_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification(텍스트 분류)은 주어진 텍스트(문장이나 문서)를 미리 정해진 카테고리(범주) 중 하나로 분류하는 작업을 의미합니다.

가장 대표적인 예시가 바로 감성 분석(Sentiment Analysis)입니다.

# 텍스트 분류란?
텍스트를 읽고, 그 텍스트가 어떤 종류에 속하는지 꼬리표(Label)를 붙이는 모든 작업을 말합니다.
예를 들어, "이 영화 정말 재밌어요!"라는 텍스트가 있다면, 텍스트 분류 모델은 이 문장을 "긍정"이라는 꼬리표로 분류합니다.

# 주요 용도 및 예시
텍스트 분류는 매우 다양한 곳에 사용됩니다.

## 감성 분석 (Sentiment Analysis):
- 목적: 문장이 긍정적인지, 부정적인지, 중립적인지 판단합니다.
- 예시: 영화 리뷰, 쇼핑몰 상품평, 소셜 미디어 반응 분석

예를 들어 "배송도 빠르고 제품도 마음에 듭니다."라는 문장은 -> (출력) 긍정(Positive)으로 분류됩니다.

## 뉴스 기사 분류 (News Categorization):
- 목적: 기사 내용을 바탕으로 카테고리를 분류합니다.
- 예시: 네이버나 다음 뉴스 섹션

예를 들어 "A 선수가 해트트릭을 기록하며 팀을 승리로 이끌었습니다..."라는 문장은 -> (출력) 스포츠(Sports)로 분류됩니다.

## 스팸 메일 필터링 (Spam Detection):
- 목적: 이메일 내용이 정상 메일인지 스팸(광고) 메일인지 구분합니다.
- 예시: Gmail, 네이버 메일의 스팸함

예를 들어 "(광고) 특별 할인! 지금 바로 확인하세요..."라는 문장은 -> (출력) 스팸(Spam)으로 분류됩니다.

## 주제 분류 (Topic Classification):
- 목적: 고객센터 문의 내용이나 게시판 글의 주제를 분류합니다.
- 예시: "로그인이 안 돼요" → 계정 문의, "결제 취소해 주세요" → 결제 문의

Huggingface에 있는 모델을 가지고 위에선 언급된 항목별로 샘플 테스트를 합니다.

샘플 코드를 살펴보시죠.

In [ ]:
# huggingface transformers 설치
!pip install transformers

# 감정 분석 (Sentiment Analysis):

In [ ]:
# import library
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# load model
tokenizer = AutoTokenizer.from_pretrained("Copycats/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("Copycats/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

# target reviews
review_list = [
    "음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.",
    "분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.",
    "기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.",
    "배우들 연기는 정말 최고였어요.",
    '한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?',
    "오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.",
    "영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다."
]

# predict
for idx, review in enumerate(review_list):
  pred = sentiment_classifier(review)
  print(f'{review}\n>> {pred[0]}')


- label 0 : 부정적인 리뷰
- label 1 : 긍정적인 리뷰

테스트 문장을 긍정과 부정이 섞인 복합감정인 것으로 테스트를 하였습니다. 결과는 위의 수치와 같습니다. 조금더 자세히 살펴보면,

1. 문장의 구조 (A-but-B)
"A-but-B" ("A(긍정)이지만 B(부정)이다") 구조에서, 사람들은 보통 B에 더 강한 강조점이나 최종 결론을 둡니다.
- A(긍정): 음식 맛은 정말 훌륭한데
- B(부정): 가게가 너무 시끄럽고 직원이 불친절해요.

이 문장은 "맛은 좋았으나, 시끄럽고 불친절해서 (전반적으로) 별로였다"라는 뉘앙스를 강하게 풍깁니다.

2. 부정적인 요소의 '양'
단순히 키워드만 보더라도,
- 긍정 키워드: 훌륭한데 (1개)
- 부정 키워드: 시끄럽고, 불친절해요 (2개)

일반적인 감성 분석 모델은 문장 전체를 보고 하나의 레이블(긍정/부정)을 결정해야 할 때, 부정적인 요소가 2가지나 명확하게 언급되었기 때문에 부정으로 판단할 가능성이 큽니다.

이 관점에서 접근하면 감정 분석 결과는 훌륭하다고 볼 수 있다.